In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
%matplotlib inline
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [13]:
homePath = "data"
trainDataPath = os.path.join(homePath, "train.csv")
testDataPath = os.path.join(homePath, "test.csv")
trainData = pd.read_csv(trainDataPath)
testData = pd.read_csv(testDataPath)

In [14]:
# y = trainData['y']
# trainData.drop('y', axis=1, inplace=True)
# trainData = pd.concat([trainData, testData], axis=0, ignore_index=True)
len(trainData)

652936

In [15]:
from sklearn.cluster import KMeans
est = KMeans(n_clusters=5, init="k-means++", n_jobs=-1)
est.fit(trainData['district_id'].reshape(-1, 1))
trainData['district_id'] = est.predict(trainData['district_id'].reshape(-1, 1))
testData['district_id'] = est.predict(testData['district_id'].reshape(-1, 1))

/home/heolis/anaconda3/envs/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:3: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  This is separate from the ipykernel package so we can avoid doing imports until
/home/heolis/anaconda3/envs/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:4: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  after removing the cwd from sys.path.
/home/heolis/anaconda3/envs/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:5: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  """


In [16]:
est = KMeans(n_clusters=5, init="k-means++", n_jobs=-1)
est.fit(trainData['area_id'].reshape(-1, 1))
trainData['area_id'] = est.predict(trainData['area_id'].reshape(-1, 1))
testData['area_id'] = est.predict(testData['area_id'].reshape(-1, 1))

/home/heolis/anaconda3/envs/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:2: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  
/home/heolis/anaconda3/envs/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:3: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  This is separate from the ipykernel package so we can avoid doing imports until
/home/heolis/anaconda3/envs/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:4: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  after removing the cwd from sys.path.


In [17]:
index = trainData['floors_before'] >= trainData['floors_after']
trainData = trainData[index]

In [18]:
index = trainData['age'] <= 176
trainData = trainData[index]

In [19]:
index = trainData['height_after'] <= trainData['height_before']
trainData = trainData[index]

In [20]:
index = trainData['height_before'] < 50
trainData = trainData[index]

In [21]:
land_condition = trainData['land_condition']
foundation_type = trainData['foundation_type']
roof_type = trainData['roof_type']
ground_floor_type = trainData['ground_floor_type']

trainData = pd.get_dummies(trainData, columns=['position', 'land_condition', 'foundation_type', 'roof_type', 'ground_floor_type'])

In [22]:
land_condition.replace(['F', 'M', 'S'], [1, 2, 3], inplace=True)
foundation_type.replace(['M', 'C', 'R', 'B', 'O'], [5, 4, 3, 2, 1], inplace=True)
roof_type.replace(['L', 'H', 'R'], [3, 2, 1], inplace=True)
ground_floor_type.replace(['M', 'R', 'B', 'T', 'O'], [5, 4, 3, 2, 1], inplace=True)
trainData['4_rebuild'] = land_condition + foundation_type + roof_type + ground_floor_type
trainData['l_f'] = land_condition + foundation_type
trainData['l_r'] = land_condition + roof_type
trainData['l_g'] = land_condition + ground_floor_type
trainData['f_r'] = foundation_type + roof_type
trainData['f_g'] = foundation_type + ground_floor_type
trainData['r_g'] = roof_type + ground_floor_type

In [23]:
land_condition = testData['land_condition']
foundation_type = testData['foundation_type']
roof_type = testData['roof_type']
ground_floor_type = testData['ground_floor_type']

testData = pd.get_dummies(testData, columns=['position', 'land_condition', 'foundation_type', 'roof_type', 'ground_floor_type'])

In [24]:
land_condition.replace(['F', 'M', 'S'], [1, 2, 3], inplace=True)
foundation_type.replace(['M', 'C', 'R', 'B', 'O'], [5, 4, 3, 2, 1], inplace=True)
roof_type.replace(['L', 'H', 'R'], [3, 2, 1], inplace=True)
ground_floor_type.replace(['M', 'R', 'B', 'T', 'O'], [5, 4, 3, 2, 1], inplace=True)
testData['4_rebuild'] = land_condition + foundation_type + roof_type + ground_floor_type
testData['l_f'] = land_condition + foundation_type
testData['l_r'] = land_condition + roof_type
testData['l_g'] = land_condition + ground_floor_type
testData['f_r'] = foundation_type + roof_type
testData['f_g'] = foundation_type + ground_floor_type
testData['r_g'] = roof_type + ground_floor_type

In [25]:
trainData['per_floor_height_before'] = trainData['height_before'] / trainData['floors_before']
trainData['per_floor_height_after'] = trainData['height_after'] / trainData['floors_after']

In [26]:
testData['per_floor_height_before'] = testData['height_before'] / testData['floors_before']
testData['per_floor_height_after'] = testData['height_after'] / testData['floors_after']

In [27]:
trainData["age_area"] = trainData['age'] / trainData['area']
testData["age_area"] = testData['age'] / testData['area']

In [28]:
trainData.fillna(0, inplace=True)

In [29]:
trainData.drop('id', axis=1, inplace=True)

In [30]:
testData.fillna(0, inplace=True)
testData.drop('id', axis=1, inplace=True)

import lightgbm as lgb

params = {
    'learning_rate':0.1,
    'lambda_l1':0.1,
    'lambda_l2':0.2,
    'max_depth':4,
    'objective':'multiclass',
    'num_class':4
}

lgb_train = lgb.Dataset(train[:600000], y[:600000])
lgb_eval = lgb.Dataset(train[600000:], y[600000:])
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=20,
                valid_sets=lgb_eval,
                early_stopping_rounds=5)
lgb.plot_importance(gbm, figsize=(10,10))

In [31]:
corr = trainData.corr()
corr['y'].sort_values()

per_floor_height_after     -0.517127
height_after               -0.443536
floors_after               -0.405705
ground_floor_type_R        -0.382114
roof_type_R                -0.331644
foundation_type_R          -0.314671
foundation_type_B          -0.205903
area_id                    -0.175130
foundation_type_C          -0.172373
area                       -0.149299
per_floor_height_before    -0.146806
district_id                -0.085735
position_Not attached      -0.049879
foundation_type_O          -0.030112
land_condition_F           -0.023559
ground_floor_type_O        -0.022835
ground_floor_type_T        -0.016830
position_Attached-2 side   -0.012019
ground_floor_type_B         0.002914
land_condition_M            0.016435
position_Attached-3 side    0.017995
land_condition_S            0.018032
position_Attached-1 side    0.058592
roof_type_H                 0.082415
height_before               0.094980
roof_type_L                 0.097213
l_g                         0.156026
l

In [32]:
def test_score(y1, y2, trueLabels):
    pred_score = (y1 == trueLabels).sum() / len(trueLabels)
    pred_score += (y2 == trueLabels).sum() * 0.5 / len(trueLabels)
    return pred_score

In [33]:
train = trainData.copy()

y = train['y']
train.drop('y', axis=1, inplace=True)

# from sklearn.decomposition import PCA
# pca = PCA(n_components=14)
# pca.fit(train)
# print(pca.n_components_, pca.explained_variance_ratio_)
# train = pca.transform(train)
# test = pca.transform(testData)

In [34]:
test = testData

## XGBoost

In [67]:
import xgboost as xgb
xgb_model = xgb.XGBClassifier(objective='multi:softmax',
                              eval_metric=['map@2', 'merror'],
                              n_estimators=700,
                              num_class=4,
                              silent=1,
                              max_depth=6,
                              nthread=4,
                              learning_rate=0.1,
                              gamma=0.5,
                              min_child_weight=0.6,
                              max_delta_step=0.1,
                              subsample=0.6,
                              colsample_bytree=0.7,
                              reg_lambda=0.4,
                              reg_alpha=0.8,
                              num_leaves=250,
                              early_stopping_rounds=20,
                              num_boost_round=8000,
                              scale_pos_weight=1)

In [68]:
xgb_model.fit(train, y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.7, early_stopping_rounds=20,
       eval_metric=['map@2', 'merror'], gamma=0.5, learning_rate=0.1,
       max_delta_step=0.1, max_depth=6, min_child_weight=0.6, missing=None,
       n_estimators=700, n_jobs=1, nthread=4, num_boost_round=8000,
       num_class=4, num_leaves=250, objective='multi:softprob',
       random_state=0, reg_alpha=0.8, reg_lambda=0.4, scale_pos_weight=1,
       seed=None, silent=1, subsample=0.6)

In [69]:
pb = xgb_model.predict_proba(train)
pb = np.array(pb)
submit = pd.DataFrame()
submit['y1'] = pb.argsort()[np.arange(len(pb)), -1]
submit['y2'] = pb.argsort()[np.arange(len(pb)), -2]
print(test_score(submit['y1'].values, submit['y2'].values, y))

0.774950502878


In [79]:
pb = xgb_model.predict_proba(test)
pb = np.array(pb)
xgb_submit = pd.DataFrame()
xgb_submit['y1'] = pb.argsort()[np.arange(len(pb)), -1]
xgb_submit['y2'] = pb.argsort()[np.arange(len(pb)), -2]

## LightGBM

In [72]:
import lightgbm as lgb
# create dataset for lightgbm  
lgb_train = lgb.Dataset(train[:600000], y[:600000])  
lgb_eval = lgb.Dataset(train[600000:], y[600000:], reference=lgb_train)  
# specify your configurations as a dict  
params = {  
    'boosting_type': 'gbdt',  
    'objective': 'multiclass',  
    'num_class': 4,  
    'metric': ['multi_error', 'map@2'],  # 'map@2', 
    'num_leaves': 250, # 4
    'min_data_in_leaf': 100,
    'learning_rate': 0.1,  
#     'feature_fraction': 0.3,  
    'bagging_fraction': 0.8,  
    'bagging_freq': 5,  
    'lambda_l1': 0.4,  
    'lambda_l2': 0.6,
    'max_depth':6,
#     'min_gain_to_split': 0.2,  
    'verbose': 5,  
    'is_unbalance': True
}  
  
print('Start training...')  
gbm = lgb.train(params,  
                lgb_train,  
                num_boost_round=8000,  
                valid_sets=lgb_eval,  
                early_stopping_rounds=500)  
  

Start training...
[1]	valid_0's multi_error: 0.347411
Training until validation scores don't improve for 500 rounds.
[2]	valid_0's multi_error: 0.346367
[3]	valid_0's multi_error: 0.345833
[4]	valid_0's multi_error: 0.345744
[5]	valid_0's multi_error: 0.3455
[6]	valid_0's multi_error: 0.3455
[7]	valid_0's multi_error: 0.344677
[8]	valid_0's multi_error: 0.344077
[9]	valid_0's multi_error: 0.344033
[10]	valid_0's multi_error: 0.343455
[11]	valid_0's multi_error: 0.342921
[12]	valid_0's multi_error: 0.343032
[13]	valid_0's multi_error: 0.342543
[14]	valid_0's multi_error: 0.342721
[15]	valid_0's multi_error: 0.342721
[16]	valid_0's multi_error: 0.342232
[17]	valid_0's multi_error: 0.341965
[18]	valid_0's multi_error: 0.341099
[19]	valid_0's multi_error: 0.34121
[20]	valid_0's multi_error: 0.340432
[21]	valid_0's multi_error: 0.340676
[22]	valid_0's multi_error: 0.340476
[23]	valid_0's multi_error: 0.340432
[24]	valid_0's multi_error: 0.340254
[25]	valid_0's multi_error: 0.339498
[26]	val

[218]	valid_0's multi_error: 0.328873
[219]	valid_0's multi_error: 0.328806
[220]	valid_0's multi_error: 0.328895
[221]	valid_0's multi_error: 0.328851
[222]	valid_0's multi_error: 0.328873
[223]	valid_0's multi_error: 0.328806
[224]	valid_0's multi_error: 0.328984
[225]	valid_0's multi_error: 0.328939
[226]	valid_0's multi_error: 0.328917
[227]	valid_0's multi_error: 0.328717
[228]	valid_0's multi_error: 0.328828
[229]	valid_0's multi_error: 0.328917
[230]	valid_0's multi_error: 0.328895
[231]	valid_0's multi_error: 0.328717
[232]	valid_0's multi_error: 0.329095
[233]	valid_0's multi_error: 0.328895
[234]	valid_0's multi_error: 0.328873
[235]	valid_0's multi_error: 0.328739
[236]	valid_0's multi_error: 0.328717
[237]	valid_0's multi_error: 0.328873
[238]	valid_0's multi_error: 0.328895
[239]	valid_0's multi_error: 0.328784
[240]	valid_0's multi_error: 0.328762
[241]	valid_0's multi_error: 0.328917
[242]	valid_0's multi_error: 0.329073
[243]	valid_0's multi_error: 0.328939
[244]	valid_

[435]	valid_0's multi_error: 0.326983
[436]	valid_0's multi_error: 0.327117
[437]	valid_0's multi_error: 0.32705
[438]	valid_0's multi_error: 0.327028
[439]	valid_0's multi_error: 0.327206
[440]	valid_0's multi_error: 0.327117
[441]	valid_0's multi_error: 0.326983
[442]	valid_0's multi_error: 0.327139
[443]	valid_0's multi_error: 0.327317
[444]	valid_0's multi_error: 0.32725
[445]	valid_0's multi_error: 0.327295
[446]	valid_0's multi_error: 0.327117
[447]	valid_0's multi_error: 0.327206
[448]	valid_0's multi_error: 0.327295
[449]	valid_0's multi_error: 0.327317
[450]	valid_0's multi_error: 0.327339
[451]	valid_0's multi_error: 0.327317
[452]	valid_0's multi_error: 0.327428
[453]	valid_0's multi_error: 0.32745
[454]	valid_0's multi_error: 0.327295
[455]	valid_0's multi_error: 0.327339
[456]	valid_0's multi_error: 0.327272
[457]	valid_0's multi_error: 0.327406
[458]	valid_0's multi_error: 0.327317
[459]	valid_0's multi_error: 0.327183
[460]	valid_0's multi_error: 0.32705
[461]	valid_0's 

[652]	valid_0's multi_error: 0.326717
[653]	valid_0's multi_error: 0.326806
[654]	valid_0's multi_error: 0.326761
[655]	valid_0's multi_error: 0.326761
[656]	valid_0's multi_error: 0.326561
[657]	valid_0's multi_error: 0.326672
[658]	valid_0's multi_error: 0.32665
[659]	valid_0's multi_error: 0.326694
[660]	valid_0's multi_error: 0.326806
[661]	valid_0's multi_error: 0.326717
[662]	valid_0's multi_error: 0.326539
[663]	valid_0's multi_error: 0.326561
[664]	valid_0's multi_error: 0.326672
[665]	valid_0's multi_error: 0.32665
[666]	valid_0's multi_error: 0.326628
[667]	valid_0's multi_error: 0.326494
[668]	valid_0's multi_error: 0.326517
[669]	valid_0's multi_error: 0.326717
[670]	valid_0's multi_error: 0.326694
[671]	valid_0's multi_error: 0.326672
[672]	valid_0's multi_error: 0.326761
[673]	valid_0's multi_error: 0.326983
[674]	valid_0's multi_error: 0.327072
[675]	valid_0's multi_error: 0.327183
[676]	valid_0's multi_error: 0.327117
[677]	valid_0's multi_error: 0.327161
[678]	valid_0'

[869]	valid_0's multi_error: 0.326094
[870]	valid_0's multi_error: 0.326005
[871]	valid_0's multi_error: 0.326116
[872]	valid_0's multi_error: 0.325983
[873]	valid_0's multi_error: 0.32605
[874]	valid_0's multi_error: 0.326005
[875]	valid_0's multi_error: 0.32605
[876]	valid_0's multi_error: 0.326005
[877]	valid_0's multi_error: 0.325961
[878]	valid_0's multi_error: 0.325805
[879]	valid_0's multi_error: 0.32585
[880]	valid_0's multi_error: 0.32585
[881]	valid_0's multi_error: 0.325716
[882]	valid_0's multi_error: 0.32565
[883]	valid_0's multi_error: 0.325694
[884]	valid_0's multi_error: 0.325761
[885]	valid_0's multi_error: 0.325761
[886]	valid_0's multi_error: 0.325672
[887]	valid_0's multi_error: 0.32585
[888]	valid_0's multi_error: 0.32605
[889]	valid_0's multi_error: 0.326161
[890]	valid_0's multi_error: 0.32605
[891]	valid_0's multi_error: 0.32605
[892]	valid_0's multi_error: 0.325939
[893]	valid_0's multi_error: 0.326139
[894]	valid_0's multi_error: 0.326139
[895]	valid_0's multi

[1083]	valid_0's multi_error: 0.324916
[1084]	valid_0's multi_error: 0.324938
[1085]	valid_0's multi_error: 0.324983
[1086]	valid_0's multi_error: 0.325072
[1087]	valid_0's multi_error: 0.325005
[1088]	valid_0's multi_error: 0.325072
[1089]	valid_0's multi_error: 0.325116
[1090]	valid_0's multi_error: 0.324983
[1091]	valid_0's multi_error: 0.324961
[1092]	valid_0's multi_error: 0.324938
[1093]	valid_0's multi_error: 0.324961
[1094]	valid_0's multi_error: 0.324872
[1095]	valid_0's multi_error: 0.324849
[1096]	valid_0's multi_error: 0.324849
[1097]	valid_0's multi_error: 0.324938
[1098]	valid_0's multi_error: 0.324983
[1099]	valid_0's multi_error: 0.324916
[1100]	valid_0's multi_error: 0.325049
[1101]	valid_0's multi_error: 0.324983
[1102]	valid_0's multi_error: 0.325072
[1103]	valid_0's multi_error: 0.325072
[1104]	valid_0's multi_error: 0.325116
[1105]	valid_0's multi_error: 0.324938
[1106]	valid_0's multi_error: 0.325138
[1107]	valid_0's multi_error: 0.325183
[1108]	valid_0's multi_er

[1294]	valid_0's multi_error: 0.325783
[1295]	valid_0's multi_error: 0.325783
[1296]	valid_0's multi_error: 0.325827
[1297]	valid_0's multi_error: 0.325694
[1298]	valid_0's multi_error: 0.325694
[1299]	valid_0's multi_error: 0.325583
[1300]	valid_0's multi_error: 0.325561
[1301]	valid_0's multi_error: 0.325538
[1302]	valid_0's multi_error: 0.32565
[1303]	valid_0's multi_error: 0.325583
[1304]	valid_0's multi_error: 0.325494
[1305]	valid_0's multi_error: 0.32545
[1306]	valid_0's multi_error: 0.32545
[1307]	valid_0's multi_error: 0.325494
[1308]	valid_0's multi_error: 0.325605
[1309]	valid_0's multi_error: 0.325605
[1310]	valid_0's multi_error: 0.325672
[1311]	valid_0's multi_error: 0.325583
[1312]	valid_0's multi_error: 0.325605
[1313]	valid_0's multi_error: 0.325516
[1314]	valid_0's multi_error: 0.325627
[1315]	valid_0's multi_error: 0.325494
[1316]	valid_0's multi_error: 0.325561
[1317]	valid_0's multi_error: 0.325494
[1318]	valid_0's multi_error: 0.325672
[1319]	valid_0's multi_error

[1505]	valid_0's multi_error: 0.324783
[1506]	valid_0's multi_error: 0.324849
[1507]	valid_0's multi_error: 0.324938
[1508]	valid_0's multi_error: 0.324983
[1509]	valid_0's multi_error: 0.325094
[1510]	valid_0's multi_error: 0.325094
[1511]	valid_0's multi_error: 0.324938
[1512]	valid_0's multi_error: 0.324649
[1513]	valid_0's multi_error: 0.324583
[1514]	valid_0's multi_error: 0.324583
[1515]	valid_0's multi_error: 0.324583
[1516]	valid_0's multi_error: 0.324427
[1517]	valid_0's multi_error: 0.324294
[1518]	valid_0's multi_error: 0.324116
[1519]	valid_0's multi_error: 0.324316
[1520]	valid_0's multi_error: 0.324249
[1521]	valid_0's multi_error: 0.324294
[1522]	valid_0's multi_error: 0.324472
[1523]	valid_0's multi_error: 0.324427
[1524]	valid_0's multi_error: 0.324583
[1525]	valid_0's multi_error: 0.32456
[1526]	valid_0's multi_error: 0.324583
[1527]	valid_0's multi_error: 0.324738
[1528]	valid_0's multi_error: 0.324694
[1529]	valid_0's multi_error: 0.324783
[1530]	valid_0's multi_err

[1716]	valid_0's multi_error: 0.324538
[1717]	valid_0's multi_error: 0.324338
[1718]	valid_0's multi_error: 0.324227
[1719]	valid_0's multi_error: 0.32436
[1720]	valid_0's multi_error: 0.324383
[1721]	valid_0's multi_error: 0.324427
[1722]	valid_0's multi_error: 0.324383
[1723]	valid_0's multi_error: 0.32436
[1724]	valid_0's multi_error: 0.324383
[1725]	valid_0's multi_error: 0.324338
[1726]	valid_0's multi_error: 0.324449
[1727]	valid_0's multi_error: 0.324494
[1728]	valid_0's multi_error: 0.324449
[1729]	valid_0's multi_error: 0.324383
[1730]	valid_0's multi_error: 0.324271
[1731]	valid_0's multi_error: 0.32436
[1732]	valid_0's multi_error: 0.32436
[1733]	valid_0's multi_error: 0.324294
[1734]	valid_0's multi_error: 0.324338
[1735]	valid_0's multi_error: 0.32436
[1736]	valid_0's multi_error: 0.324672
[1737]	valid_0's multi_error: 0.324783
[1738]	valid_0's multi_error: 0.324649
[1739]	valid_0's multi_error: 0.324716
[1740]	valid_0's multi_error: 0.32476
[1741]	valid_0's multi_error: 0

[1927]	valid_0's multi_error: 0.324827
[1928]	valid_0's multi_error: 0.324894
[1929]	valid_0's multi_error: 0.324872
[1930]	valid_0's multi_error: 0.324849
[1931]	valid_0's multi_error: 0.324872
[1932]	valid_0's multi_error: 0.324894
[1933]	valid_0's multi_error: 0.324827
[1934]	valid_0's multi_error: 0.324805
[1935]	valid_0's multi_error: 0.324872
[1936]	valid_0's multi_error: 0.324672
[1937]	valid_0's multi_error: 0.324649
[1938]	valid_0's multi_error: 0.324694
[1939]	valid_0's multi_error: 0.324672
[1940]	valid_0's multi_error: 0.324694
[1941]	valid_0's multi_error: 0.324672
[1942]	valid_0's multi_error: 0.324605
[1943]	valid_0's multi_error: 0.324672
[1944]	valid_0's multi_error: 0.324694
[1945]	valid_0's multi_error: 0.324649
[1946]	valid_0's multi_error: 0.324583
[1947]	valid_0's multi_error: 0.324649
[1948]	valid_0's multi_error: 0.324672
[1949]	valid_0's multi_error: 0.324738
[1950]	valid_0's multi_error: 0.324672
[1951]	valid_0's multi_error: 0.324672
[1952]	valid_0's multi_er

In [73]:
print('Start predicting...')
pb = gbm.predict(train, num_iteration=gbm.best_iteration)
pb = np.array(pb)
submit = pd.DataFrame()
submit['y1'] = pb.argsort()[np.arange(len(pb)), -1]
submit['y2'] = pb.argsort()[np.arange(len(pb)), -2]
print(test_score(submit['y1'].values, submit['y2'].values, y))

Start predicting...
0.796050152949


In [74]:
pb = gbm.predict(test, num_iteration=gbm.best_iteration)
pb = np.array(pb)
lgb_submit = pd.DataFrame()
lgb_submit['y1'] = pb.argsort()[np.arange(len(pb)), -1]
lgb_submit['y2'] = pb.argsort()[np.arange(len(pb)), -2]

In [75]:
lgb_submit.to_csv('lgb_submit.csv', index=False)

## NN

In [83]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
enc.fit(y.reshape(-1, 1))
y_hot = enc.transform(y.reshape(-1, 1))


#构建LM神经网络模型
from keras.models import Sequential #导入神经网络初始化函数
from keras.layers.core import Dense, Activation #导入神经网络层函数、激活函数
from keras.layers import Dropout
from keras.metrics import top_k_categorical_accuracy
from keras.callbacks import EarlyStopping
netfile = './net.model' #构建的神经网络模型存储路径

def acc_top2(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=2)

net = Sequential()
net.add(Dense(input_dim = 38, output_dim = 128))
net.add(Activation('relu'))
net.add(Dense(input_dim = 128, output_dim = 256))
net.add(Activation('relu'))
net.add(Dense(input_dim = 256, output_dim = 256))
net.add(Activation('relu'))
net.add(Dropout(0.3))
net.add(Dense(input_dim = 256, output_dim = 512))
net.add(Activation('relu'))
net.add(Dense(input_dim = 512, output_dim = 4))
net.add(Activation('softmax'))
net.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics=['accuracy']) # accuracy
early_stopping = EarlyStopping(monitor='val_loss', patience=50, verbose=2)

net.fit(train, y_hot, epochs=150, batch_size=4096, validation_data=(train[600000:], y_hot[600000:]), callbacks=[early_stopping])
net.save_weights(netfile) #保存模型

/home/heolis/anaconda3/envs/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:3: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  This is separate from the ipykernel package so we can avoid doing imports until
/home/heolis/anaconda3/envs/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:4: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  after removing the cwd from sys.path.
Using TensorFlow backend.
/home/heolis/anaconda3/envs/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:19: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=128, input_dim=38)`
/home/heolis/anaconda3/envs/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:21: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=256, input_dim=128)`
/home/heolis/anaconda3/envs/tensorflow/lib/python3.5/site-package

Train on 644987 samples, validate on 44987 samples
Epoch 1/150
644987/644987 [==============================] - 3s 5us/step - loss: 1.9309 - acc: 0.5842 - val_loss: 0.7911 - val_acc: 0.6194
Epoch 2/150
644987/644987 [==============================] - 2s 4us/step - loss: 0.7803 - acc: 0.6259 - val_loss: 0.7646 - val_acc: 0.6296
Epoch 3/150
644987/644987 [==============================] - 2s 4us/step - loss: 0.7697 - acc: 0.6298 - val_loss: 0.7565 - val_acc: 0.6330
Epoch 4/150
644987/644987 [==============================] - 2s 4us/step - loss: 0.7647 - acc: 0.6318 - val_loss: 0.7650 - val_acc: 0.6272
Epoch 5/150
644987/644987 [==============================] - 2s 4us/step - loss: 0.7630 - acc: 0.6324 - val_loss: 0.7562 - val_acc: 0.6338
Epoch 6/150
644987/644987 [==============================] - 2s 4us/step - loss: 0.7621 - acc: 0.6327 - val_loss: 0.7646 - val_acc: 0.6237
Epoch 7/150
644987/644987 [==============================] - 2s 4us/step - loss: 0.7615 - acc: 0.6329 - val_loss: 0

644987/644987 [==============================] - 2s 4us/step - loss: 0.7064 - acc: 0.6708 - val_loss: 0.7030 - val_acc: 0.6743
Epoch 118/150
644987/644987 [==============================] - 2s 4us/step - loss: 0.7054 - acc: 0.6709 - val_loss: 0.7016 - val_acc: 0.6776
Epoch 119/150
644987/644987 [==============================] - 2s 4us/step - loss: 0.7054 - acc: 0.6710 - val_loss: 0.7034 - val_acc: 0.6744
Epoch 120/150
644987/644987 [==============================] - 2s 4us/step - loss: 0.7053 - acc: 0.6709 - val_loss: 0.7002 - val_acc: 0.6759
Epoch 121/150
644987/644987 [==============================] - 2s 4us/step - loss: 0.7052 - acc: 0.6717 - val_loss: 0.7012 - val_acc: 0.6759
Epoch 122/150
644987/644987 [==============================] - 2s 4us/step - loss: 0.7053 - acc: 0.6714 - val_loss: 0.7003 - val_acc: 0.6783
Epoch 123/150
644987/644987 [==============================] - 2s 4us/step - loss: 0.7046 - acc: 0.6713 - val_loss: 0.6997 - val_acc: 0.6768
Epoch 124/150
644987/644987

In [84]:
predict_prob = net.predict_proba(train[600000:])
pb = np.array(predict_prob)
submit = pd.DataFrame()
submit['y1'] = pb.argsort()[np.arange(len(pb)), -1]
submit['y2'] = pb.argsort()[np.arange(len(pb)), -2]
print(test_score(submit['y1'].values, submit['y2'].values, y[600000:]))

0.775790784004


In [85]:
predict_prob = net.predict_proba(test)
pb = np.array(predict_prob)
nn_submit = pd.DataFrame()
nn_submit['y1'] = pb.argsort()[np.arange(len(pb)), -1]
nn_submit['y2'] = pb.argsort()[np.arange(len(pb)), -2]

In [107]:
xgb_submit.to_csv('xgb_subumit.csv', index=False)
lgb_submit.to_csv('lgb_submit.csv', index=False)
nn_submit.to_csv('nn_submit.csv', index=False)

In [102]:
def wsubmit(xg, lg, nn):
    xg_y1 = xg['y1'].values
    lg_y1 = lg['y1'].values
    lg_y2 = lg['y2'].values
    nn_y1 = lg['y1'].values
    submitData = pd.DataFrame()
    y1 = []
    y2 = []
    for i in range(len(xg)):
        row_y1 = [xg_y1[i], lg_y1[i], nn_y1[i]]
        y1.append(max(row_y1, key=row_y1.count))
        if max(row_y1, key=row_y1.count) != lg_y1[i]:
            y2.append(lg_y1[i])
        else:
            y2.append(lg_y2[i])
    submitData['y1'] = y1
    submitData['y2'] = y2
    submitData.to_csv('submit_voting.csv', index=False)

In [103]:
wsubmit(xgb_submit, lgb_submit, nn_submit)

## 混合

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.2, random_state=0)##test_size测试集合所占比例
##X_train_1用于生成模型  X_train_2用于和新特征组成新训练集合
X_train_1, X_train_2, y_train_1, y_train_2 = train_test_split(X_train, y_train, test_size=0.7, random_state=0)

In [ ]:
def mergeToOne(X,X2):
    return np.hstack((X, X2))

In [ ]:
import lightgbm as lgb
from xgboost.sklearn import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
xgb = XGBClassifier(booster='gbtree', 
                    learning_rate =0.1,
                    objective='multi:softmax', 
                    num_class=4, 
                    gamma=0.05, 
                    subsample=0.4, 
                    reg_alpha=1e-05,
                    n_estimators=50,
                    metric='multi_logloss',
                    colsample_bytree=0.7, 
                    silent=1, 
                    nthread=4)

gbm = lgb.LGBMClassifier(learning_rate=0.1, 
                   boosting_type='gbdt', 
                   objective='multiclass',
                   n_estimators=50,
                   metric='multi_logloss', 
                   max_depth=7, 
                   bagging_fraction=0.7, 
                   is_unbalance=True)

rf = RandomForestClassifier(n_estimators=50, 
                            min_samples_split=90, 
                            min_samples_leaf=15,
                            max_depth=8,
                            oob_score=True)

In [ ]:
xgb.fit(X_train_1, y_train_1)
new_feature= xgb.apply(X_train_2)

X_train_new2 = mergeToOne(X_train_2,new_feature)
new_feature_test = xgb.apply(X_test)
X_test_new = mergeToOne(X_test,new_feature_test)
# real_test_xgb = xgb.apply(test)
# real_test = mergeToOne(test, real_test_xgb)

gbm.fit(X_train_1, y_train_1)
new_feature = gbm.apply(X_train_2)

X_train_new2 = mergeToOne(X_train_new2,new_feature)
new_feature_test = gbm.apply(X_test)
X_test_new = mergeToOne(X_test_new,new_feature_test)
# real_test_lgb = gbm.apply(test)
# real_test = mergeToOne(real_test, real_test_lgb)

rf.fit(X_train_1, y_train_1)
new_feature = rf.apply(X_train_2)
X_train_new2 = mergeToOne(X_train_new2, new_feature)
new_feature_test = rf.apply(X_test)
X_test_new = mergeToOne(X_test_new, new_feature_test)
# real_test_lgb = rf.apply(test)
# real_test = mergeToOne(real_test, real_test_lgb)

In [ ]:
X_train_new2.shape

In [ ]:
X_train_new2.shape

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
# enc.fit(y.reshape(-1, 1))
# y = enc.transform(y.reshape(-1, 1))
enc.fit(y_train_2.reshape(-1, 1))
y_train_2 = enc.transform(y_train_2.reshape(-1, 1)).toarray()

In [ ]:
#构建LM神经网络模型
from keras.models import Sequential #导入神经网络初始化函数
from keras.layers.core import Dense, Activation #导入神经网络层函数、激活函数
from keras.layers import Dropout
from keras.callbacks import EarlyStopping
netfile = './net.model' #构建的神经网络模型存储路径


net = Sequential()
net.add(Dense(input_dim = 472, output_dim = 128))
net.add(Activation('relu'))
net.add(Dense(input_dim = 128, output_dim = 256))
net.add(Activation('relu'))
net.add(Dense(input_dim = 256, output_dim = 256))
net.add(Activation('relu'))
net.add(Dropout(0.3))
net.add(Dense(input_dim = 256, output_dim = 512))
net.add(Activation('relu'))
net.add(Dense(input_dim = 512, output_dim = 4))
net.add(Activation('softmax'))
net.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=50, verbose=2)
net.fit(X_train_new2, y_train_2, epochs=100, batch_size=4096, validation_data=(X_test_new, y_test), callbacks=[early_stopping])
# net.fit(train, y, epochs=100, batch_size=4096)
net.save_weights(netfile) #保存模型

In [ ]:
predict_prob = net.predict_proba(X_test_new)
pb = np.array(predict_prob)
submit = pd.DataFrame()
submit['y1'] = pb.argsort()[np.arange(len(pb)), -1]
submit['y2'] = pb.argsort()[np.arange(len(pb)), -2]
print(test_score(submit['y1'].values, submit['y2'].values, y_test.values))